In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from dash import callback_context

from CRUD import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "acc_user"
password = "user"
shelter = AnimalShelter(username, password)
                                                     
# class read method
df = pd.DataFrame.from_records(shelter.read_all({}))
                                               
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Brandon Stultz - SNHU CS-340 - Dashboard'))),
    html.Hr(),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
 
#--------------------------FILTER OPTIONS--- -------------------------  
    html.Div([
        dcc.RadioItems(id='filter-id',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Wilderness Rescue', 'value': 'wilderness'},
                {'label': 'Disater Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ]
        ),
    ]),
#---------------------------------------------------------------------    
    
    html.Hr(),
    
#-------------------------------DATA TABLE----------------------------  
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
            editable=True,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            column_selectable="single",
            row_selectable="single",
            row_deletable=True,
            selected_columns=[],
            selected_rows=[],
            page_action="native",
            page_current= 0,
            page_size= 10,
        
    ),
#---------------------------------GRAPH & MAP-----------------------------------
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################       
 
#-----------------------------------------FILTER LOGIC--------------------------------------    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-id', 'value')])
def update_dashboard(filter_type):

    if(filter_type == 'water'):
        df = pd.DataFrame.from_records(shelter.read_all(
            {"animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever Mix",
                            "Chesapeake Bay Retriever",
                            "Newfoundland"
                            ]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks": {"$gt": 26,"$lt": 156}
            }))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)
        
    elif(filter_type == 'wilderness'):
        df = pd.DataFrame.from_records(shelter.read_all(
            {"animal_type":"Dog",
            "breed":{"$in":["German Shepherd", 
                            "Alaskan Malamute", 
                            "Old English Sheepdog", 
                            "Siberian Husky",
                            "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks": {"$gt": 26,"$lt": 156}
            }))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)
        
    elif(filter_type == 'disaster'):
        df = pd.DataFrame.from_records(shelter.read_all(
            {"animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher", 
                            "German Shepherd", 
                            "Golden Retriever",
                            "Bloodhound", 
                            "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks": {"$gt": 20,"$lt": 300}
            }))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)
    
    elif(filter_type == 'reset'):
        df = pd.DataFrame.from_records(shelter.read_all({}))
                                            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)
#----------------------------------------------------------------------------------------------------------------

#------------------------------------------COLUMN SELECT LOGIC---------------------------------------------------

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
#----------------------------------------------------------------------------------------------------------------

#-----------------------------------------------------GRAPH LOGIC------------------------------------------------

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def generate_charts(viewData):
    ###FIX ME ####
    
    dff = pd.DataFrame.from_dict(viewData)
   
    fig = px.pie(data_frame = dff, values = 'age_upon_outcome_in_weeks', names='breed')
    
    return dcc.Graph(figure = fig)
        
    
        
        
    
#----------------------------------------------------------------------------------------------------------------

#------------------------------------------------------MAP LOGIC-------------------------------------------------
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     ])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '600px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,2]),
                dl.Popup([
                    html.H1(dff.iloc[0,3]),
                    html.P(dff.iloc[0,3]),
                    html.P(dff.iloc[0,4]),                    
                    html.P(dff.iloc[0,12])
                ])
            ])
        ])
    ]


app